In [1]:
import DATA
import Net

In [2]:
import tensorflow as tf

In [3]:
import numpy as np

In [4]:
slim = tf.contrib.slim

In [5]:
data = DATA.CADF()
net = Net.Unet(True)

Instructions for updating:
Use the `axis` argument instead


In [6]:
global_step = tf.train.create_global_step()#创建全局变量
learning_rate = 0.01
sess = tf.Session()
variable_to_restore = tf.global_variables()
saver = tf.train.Saver(variable_to_restore)#初始化
summary_op = tf.summary.merge_all()#将之前定义的所有summary整合在一起
writer = tf.summary.FileWriter("./Summ/",sess.graph,flush_secs=60)#创建一个FileWrite的类对象，并将计算图写入文件
tf.summary.scalar('learning_rate',learning_rate)
train_step = tf.train.GradientDescentOptimizer(
    learning_rate).minimize(net.total_loss, global_step=global_step)#使用随机梯度下降算法，使参数沿着 梯度的反方向，即总损失减小的方向移动，实现更新参数
with tf.control_dependencies([train_step]):
    #此函数指定某些操作执行的依赖关系，指执行完train_step，在执行train_op；
    #而tf.no_op表示执行完 train_step, variable_averages_op 操作之后什么都不做
    train_op = tf.no_op(name='train')
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range (1001):
    xs,ys = data.batch_prepare(i)
    feed_dict = {net.images:xs,net.labels:ys} 
    if i%10==0:
        summary_str,loss_value,_,step = sess.run([summary_op,net.total_loss,
                                                        train_op,global_step],feed_dict = feed_dict)
        writer.add_summary(summary_str,step)
        print("After %d training step(s), loss on training "
                      "batch is %g." % (step, loss_value))
    else :
        summary_str,loss_value,_,step = sess.run([summary_op,net.total_loss,train_op,global_step],
                                                 feed_dict = feed_dict)
    if i % 100 == 0:
        print("After %d training step(s), loss on training "
                      "batch is %g." % (step, loss_value))
        saver.save(sess,"./save/model.ckpt",global_step)
print("training is ending")

After 0 training step(s), loss on training batch is 0.845976.
After 0 training step(s), loss on training batch is 0.845976.
After 10 training step(s), loss on training batch is 0.724855.
After 20 training step(s), loss on training batch is 0.80394.
After 30 training step(s), loss on training batch is 0.592781.
After 40 training step(s), loss on training batch is 0.715635.
After 50 training step(s), loss on training batch is 0.756856.
After 60 training step(s), loss on training batch is 0.649883.
After 70 training step(s), loss on training batch is 0.792912.
After 80 training step(s), loss on training batch is 0.627614.
After 90 training step(s), loss on training batch is 0.933661.
After 100 training step(s), loss on training batch is 0.677244.
After 100 training step(s), loss on training batch is 0.677244.
After 110 training step(s), loss on training batch is 0.660727.
After 120 training step(s), loss on training batch is 0.616166.
After 130 training step(s), loss on training batch is 

In [9]:
sess.close()